# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 93 new papers today
          42 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/41 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.06433


extracting tarball to tmp_2211.06433... done.
Retrieving document from  https://arxiv.org/e-print/2211.06436


extracting tarball to tmp_2211.06436... done.
Retrieving document from  https://arxiv.org/e-print/2211.06445


extracting tarball to tmp_2211.06445...

...

/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.07428/shen.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2211.07486...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.07565


extracting tarball to tmp_2211.07565...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06454-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06454) | **Pre-main Sequence Brackett Emitters in the APOGEE DR17 Catalog: Line  Strengths and Physical Properties of Accretion Columns**  |
|| Elliott Khilfeh, et al. -- incl., <mark>Jesús Hernández</mark> |
|*Appeared on*| *2022-11-15*|
|*Comments*| *19 pages, 9 figures, accepted to AJ*|
|**Abstract**| Very young (t $\lesssim$ 10 Myrs) stars possess strong magnetic fields that channel ionized gas from the interiors of their circumstellar discs to the surface of the star. Upon impacting the stellar surface, the shocked gas recombines and emits hydrogen spectral lines. To characterize the density and temperature of the gas within these accretion streams, we measure equivalent widths of Brackett (Br) 11-20 emission lines detected in 1101 APOGEE spectra of 326 likely pre-main sequence accretors. For sources with multiple observations, we measure median epoch-to-epoch line strength variations of 10% in Br11 and 20% in Br20. We also fit the measured line ratios to predictions of radiative transfer models by Kwan & Fischer. We find characteristic best-fit electron densities of $n_e$ = 10$^{11} - 10^{12}$ cm$^{-3}$, and excitation temperatures that are inversely correlated with electron density (from T$\sim$5000 K for $n_e \sim 10^{12}$ cm$^{-3}$, to T$\sim$12500 K at $n_e \sim 10^{11}$ cm$^{-3}$). These physical parameters are in good agreement with predictions from modelling of accretion streams that account for the hydrodynamics and radiative transfer within the accretion stream. We also present a supplementary catalog of line measurements from 9733 spectra of 4255 Brackett emission line sources in the APOGEE DR17 dataset. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.07128-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.07128) | **The Young Supernova Experiment Data Release 1 (YSE DR1): Light Curves  and Photometric Classification of 1975 Supernovae**  |
|| P. D. Aleo, et al. -- incl., <mark>V. A. Villar</mark>, <mark>M. E. Huber</mark>, <mark>D. Langeroodi</mark>, <mark>K. W. Smith</mark> |
|*Appeared on*| *2022-11-15*|
|*Comments*| *Submitted to ApJ; 63 pages; 35 figures; 10 tables*|
|**Abstract**| We present the Young Supernova Experiment Data Release 1 (YSE DR1), comprised of processed multi-color Pan-STARRS1 (PS1) griz and Zwicky Transient Facility (ZTF) gr photometry of 1975 transients with host-galaxy associations, redshifts, spectroscopic/photometric classifications, and additional data products from 2019 November 24 to 2021 December 20. YSE DR1 spans discoveries and observations from young and fast-rising supernovae (SNe) to transients that persist for over a year, with a redshift distribution reaching z~0.5. We present relative SN rates from YSE's magnitude- and volume-limited surveys, which are consistent with previously published values within estimated uncertainties for untargeted surveys. We combine YSE and ZTF data, and create multi-survey SN simulations to train the ParSNIP photometric classification algorithm; when validating our classifier on 472 spectroscopically classified YSE DR1 SNe, we achieve 82% accuracy across three SN classes (SNe Ia, II, Ib/Ic) and 90% accuracy across two SN classes (SNe Ia, core-collapse SNe). Our classifier performs particularly well on SNe Ia, with high (>90%) individual completeness and purity, which will help build an anchor photometric SNe Ia sample for cosmology. We then use our photometric classifier to characterize our photometric sample of 1483 SNe, labeling 1048 (~71%) SNe Ia, 339 (~23%) SNe II, and 96 (~6%) SNe Ib/Ic. Our approach demonstrates that simulations can be used to improve the performance of photometric classifiers applied to real data. YSE DR1 provides a training ground for building discovery, anomaly detection, and classification algorithms, performing cosmological analyses, understanding the nature of red and rare transients, exploring tidal disruption events and nuclear variability, and preparing for the forthcoming Vera C. Rubin Observatory Legacy Survey of Space and Time. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06433-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06433) | **The link between infall location, early disc size, and the fraction of  self-gravitationally fragmenting discs**  |
|| O. Schib, C. Mordasini, <mark>R. Helled</mark> |
|*Appeared on*| *2022-11-15*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Many protoplanetary discs are self-gravitating early in their lives. If they fragment under their own gravity, they form bound gaseous clumps which may evolve to become giant planets. Today, the fraction of discs that undergo fragmentation, and the frequency of conditions that may lead to giant planet formation via gravitational instability, is still unknown. We perform a population synthesis of discs from formation to dispersal. In varying the infall radius, we study the relationship of the early disc size with fragmentation. Furthermore, we investigate how stellar accretion heating affects the fragmentation fraction. We find that discs fragment only if they become sufficiently large early in their lives. This size depends sensitively on where mass is added to the discs during the collapse of their parent molecular cloud core. By choosing intermediate infall locations leading to a synthetic disc size distribution that is in agreement with the observed one, we find a fragmentation fraction between 0.1 and 11 %, depending on the efficiency of stellar accretion heating of the discs. We conclude that the early disc size is mainly determined by the infall location during the collapse of the molecular cloud core and controls the population-wide frequency of fragmentation. Stellar accretion heating plays an important role for fragmentation and must be studied further. Our work is an observationally-informed step towards a prediction of the frequency of giant planet formation by gravitational instability. Upcoming observations and theoretical studies will deepen our understanding of the formation and early evolution of discs, eventually allowing to understand how infall, disc morphology, giant planet formation via gravitational instability, and the observed extrasolar planet population are linked. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.07486-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.07486) | **Saturn's Seasonal Variability from Four Decades of Ground-Based  Mid-Infrared Observations**  |
|| James S.D. Blake, et al. -- incl., <mark>Henrik Melin</mark> |
|*Appeared on*| *2022-11-15*|
|*Comments*| *25 pages, 15 figures, accepted for publication in Icarus*|
|**Abstract**| A multi-decade record of ground-based mid-infrared (7-25 $\mu$m) images of Saturn is used to explore seasonal and non-seasonal variability in thermal emission over more than a Saturnian year (1984-2022). Thermal emission measured by 3-m and 8-m-class observatories compares favourably with synthetic images based on both Cassini-derived temperature records and the predictions of radiative climate models. 8-m class facilities are capable of resolving thermal contrasts on the scale of Saturn's belts, zones, polar hexagon, and polar cyclones, superimposed onto large-scale seasonal asymmetries. Seasonal changes in brightness temperatures of $\sim30$ K in the stratosphere and $\sim10$ K in the upper troposphere are observed, as the northern and southern polar stratospheric vortices (NPSV and SPSV) form in spring and dissipate in autumn. The timings of the first appearance of the warm polar vortices is successfully reproduced by radiative climate models, confirming them to be radiative phenomena, albeit entrained within sharp boundaries influenced by dynamics. Axisymmetric thermal bands (4-5 per hemisphere) display temperature gradients that are strongly correlated with Saturn's zonal winds, indicating winds that decay in strength with altitude, and implying meridional circulation cells forming the system of cool zones and warm belts. Saturn's thermal structure is largely repeatable from year to year (via comparison of infrared images in 1989 and 2018), with the exception of low-latitudes. Here we find evidence of inter-annual variations because the equatorial banding at 7.9 $\mu$m is inconsistent with a $\sim15$-year period for Saturn's equatorial stratospheric oscillation, i.e., it is not strictly semi-annual. Finally, observations between 2017-2022 extend the legacy of the Cassini mission, revealing the continued warming of the NPSV during northern summer. [Abr.] |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.07565-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.07565) | **Quijote-PNG: Quasi-maximum likelihood estimation of Primordial  Non-Gaussianity in the non-linear halo density field**  |
|| Gabriel Jung, et al. -- incl., <mark>Michele Liguori</mark>, <mark>Licia Verde</mark> |
|*Appeared on*| *2022-11-15*|
|*Comments*| *16 pages, 10 figures*|
|**Abstract**| We study primordial non-Gaussian signatures in the redshift-space halo field on non-linear scales, using a quasi-maximum likelihood estimator based on optimally compressed power spectrum and modal bispectrum statistics. We train and validate the estimator on a suite of halo catalogues constructed from the Quijote-PNG N-body simulations, which we release to accompany this paper. We verify its unbiasedness and near optimality, for the three main types of primordial non-Gaussianity (PNG): local, equilateral, and orthogonal. We compare the modal bispectrum expansion with a $k$-binning approach, showing that the former allows for faster convergence of numerical derivatives in the computation of the score-function, thus leading to better final constraints. We find, in agreement with previous studies, that the local PNG signal in the halo-field is dominated by the scale-dependent bias signature on large scales and saturates at $k \sim 0.2~h\,\mathrm{Mpc}^{-1}$, whereas the small-scale bispectrum is the main source of information for equilateral and orthogonal PNG. Combining power spectrum and bispectrum on non-linear scales plays an important role in breaking degeneracies between cosmological and PNG parameters; such degeneracies remain however strong for equilateral PNG. We forecast that PNG parameters can be constrained with $\Delta f_\mathrm{NL}^\mathrm{local} = 45$, $\Delta f_\mathrm{NL}^\mathrm{equil} = 570$, $\Delta f_\mathrm{NL}^\mathrm{ortho} = 110$, on a cubic volume of $1 \left({ {\rm Gpc}/{ {\rm h}}} \right)^3$, at $z = 1$, considering scales up to $k_\mathrm{max} = 0.5~\mathrm{Mpc}^{-1}$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06518-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06518) | **EOS Manual**  |
|| <mark>Li Zeng</mark>, Stein B. Jacobsen, Dimitar D. Sasselov, Michail I. Petaev |
|*Appeared on*| *2022-11-15*|
|*Comments*| *For pedagogical purpose only (non-profit). Comments and suggestions welcome!*|
|**Abstract**| The exoplanet field now abounds with new discoveries of planets and planetary systems. It is of great interest for the scientific community to understand the compositions and internal structures of these new planets outside our own solar system, and then infer their formation scenarios. In particular, the proper implementation of the equation-of-states of various matters into the calculation of internal structure is crucial. Thus, we have come up with this write-up that provides the detailed steps to turn an equation-of-state (EOS) into a mass-radius relation of exoplanets, and highlight the physics and chemistry involved in these calculations. It must be visualized! |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06450-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06450) | **Spectropolarimetry as a Means to Address Cloud Composition and  Habitability for a Cloudy Exoplanetary Atmosphere in the Habitable Zone**  |
|| Robert A. West, et al. -- incl., <mark>Philip Dumont</mark> |
|*Appeared on*| *2022-11-15*|
|*Comments*| *ApJ, accepted*|
|**Abstract**| In our solar system, the densely cloud-covered atmosphere of Venus stands out as an example of how polarimetry can be used to gain information on cloud composition and particle mean radius. With current interest running high on discovering and characterizing extrasolar planets in the habitable zone where water exists in the liquid state, making use of spectropolarimetric measurements of directly-imaged exoplanets could provide key information unobtainable through other means. In principle, spectropolarimetric measurements can determine if acidity causes water activities in the clouds to be too low for life. To this end, we show that a spectropolarimeter measurement over the range 400 nm - 1000 nm would need to resolve linear polarization to a precision of about 1% or better for reflected starlight from an optically thick cloud-enshrouded exoplanet. We assess the likelihood of achieving this goal by simulating measurements from a notional spectropolarimeter as part of a starshade configuration for a large space telescope (a HabEx design, but for a 6 m diameter primary mirror). Our simulations include consideration of noise from a variety of sources. We provide guidance on limits that would need to be levied on instrumental polarization to address the science issues we discuss. For photon-limited noise, integration times would need to be of order one hour for a large radius (10 Earth radii) planet to more than 100 hours for smaller exoplanets depending on the star-planet separation, planet radius, phase angle and desired uncertainty. We discuss implications for surface chemistry and habitability. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06942-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06942) | **V5856 Sagittarii/2016: Broad Multi-Epoch Spectral Coverage of a  Sustained High Luminosity Nova**  |
|| Robert Williams, et al. -- incl., <mark>Frederick M. Walter</mark> |
|*Appeared on*| *2022-11-15*|
|*Comments*| *27 pages; 13 figures. Accepted for publication in the ApJ*|
|**Abstract**| Nova V5856 Sagittarii is unique for having remained more than nine magnitudes above its pre-outburst brightness for more than six years. Extensive visible and IR spectra from the time of outburst to the present epoch reveal separate emitting regions with distinct spectral characteristics. Permitted emission lines have both broad and narrow components, whereas the forbidden line profiles are almost entirely broad. The permitted line components frequently display P Cygni profiles indicating high optical depth, whereas the broad components do not show detectable absorption. The densities and velocities deduced from the spectra, including differences in the O I 7773 and 8446 lines, are not consistent with an on-going wind. Instead, the prolonged high luminosity and spectral characteristics are indicative of a post-outburst common envelope that enshrouds the binary, and is likely the primary source of the visible and IR emission. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.06454.md
    + _build/html/tmp_2211.06454/./9250-58037-022.png
    + _build/html/tmp_2211.06454/./10083-58057-200.png
    + _build/html/tmp_2211.06454/./7220-56606-227.png
    + _build/html/tmp_2211.06454/./9533-58063-189.png
    + _build/html/tmp_2211.06454/./mdot.png
    + _build/html/tmp_2211.06454/./Br11_Plot9.png
    + _build/html/tmp_2211.06454/./br11.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.06454-b31b1b.svg)](https://arxiv.org/abs/2211.06454)<mark>Appeared on: 2022-11-15</mark> - _19 pages, 9 figures, accepted to AJ_

</div>
<div id="authors">

Elliott Khilfeh, et al. -- incl., <mark><mark>Jesús Hernández</mark></mark>

</div>
<div id="abstract">

**Abstract:** Very young (t$\lesssim$10 Myrs) stars possess strong magnetic fields that channel ionized gas from the interiors of their circumstellar discs to the surface of the star. Upon impacting the stellar surface, the shocked gas recombines and emits hydrogen spectral lines. To characterize the density and temperature of the gas within these accretion streams, we measure equivalent widths of Brackett (Br) 11 – 20 emission lines detected in 1101 APOGEE spectra of 326 likely pre-main sequence accretors. For sources with multiple observations, we measure median epoch-to-epoch line strength variations of 10\%in Br11 and 20\%in Br20. We also fit the measured line ratios to predictions of radiative transfer models by Kwan\&Fischer.  We find characteristic best-fit electron densities of$n_e$= 10$^{11} - 10^{12}$cm$^{-3}$, and excitation temperatures that are inversely correlated with electron density (from T$\sim$5000 K for$n_e \sim 10^{12}$cm$^{-3}$, to T$\sim$12500 K at$n_e \sim 10^{11}$cm$^{-3}$). These physical parameters are in good agreement with predictions from modelling of accretion streams that account for the hydrodynamics and radiative transfer within the accretion stream. We also present a supplementary catalog of line measurements from 9733 spectra of 4255 Brackett emission line sources in the APOGEE DR17 dataset.

</div>

<div id="div_fig1">

<img src="tmp_2211.06454/./9250-58037-022.png" alt="Fig9.1" width="25%"/><img src="tmp_2211.06454/./10083-58057-200.png" alt="Fig9.2" width="25%"/><img src="tmp_2211.06454/./7220-56606-227.png" alt="Fig9.3" width="25%"/><img src="tmp_2211.06454/./9533-58063-189.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** \textit{(top left panel)}: Example of a source with (double-peaked) Br 11 emission, and a neighboring 16781 line with an identical velocity profile. These sources are coded with an FeII flag = 1 in the machine readable table of our sample.  \textit{(top right panel)}: Example of a source with strong Br 11 emission, and two lines near the wavelength expected for the neighboring [FeII] line. These sources are coded with an FeII flag = 2 in the machine readable table of our sample. \textit{(Bottom panels)}: Examples of two sources with a prominent emission feature at 16775: the source on the left shows this line near two photospheric features, while the source on the right shows two additional emission lines in between the 16775 line and Br11.  Both of these patterns which include the 16775 feature are coded with an FeII flag = 3 in the machine readable table of our sample. \label{fig:FeII_emitters} (*fig:FeII_emitters*)

</div>
<div id="div_fig2">

<img src="tmp_2211.06454/./mdot.png" alt="Fig1" width="100%"/>

**Figure 1. -** Accretion rate measurements for sources with optical measurements by \citet{Manzo-Martinez2020}(full sample shown as the blue histogram), highlighting the subsets of sources with APOGEEnet stellar parameters (yellow histogram), and with Brackett emission lines detected in this work (black histogram). The Brackett sample is strongly biased towards the most rapidly accreting sources in the \citet{Manzo-Martinez2020} sample, given the larger temperatures and densities required to populate hydrogen's n=11-20 levels, and the larger optical depths that are necessary to produce detectable emission lines. \label{fig:Manzo_comparison} (*fig:Manzo_comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2211.06454/./Br11_Plot9.png" alt="Fig6.1" width="50%"/><img src="tmp_2211.06454/./br11.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Left: Wavelength-flux plot of the Br11 emission line for a strongly accreting YSO, 2M05471411+0009073. The green highlighted regions are the windows which were used to calculate the continuum (horizontal dashed line) on the left and right of the central line (vertical dashed line). The equivalent width of the Br11 line is given by the width of the shaded blue region below the continuum. Right: Distribution of the absolute value of equivalent widths for Br11 for all of the objects in the sample, excluding those that are likely associated with Be stars or that have nebular emission. Note that since Br11 line is seen in emission, by convention EqW is negative. \label{fig:EqW} (*fig:EqW*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

88  publications in the last 7 days.
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers